# Auto Insurance Premiums Prediction
### By Samuel Mendez

### Objective:
Develop a model to predict the insurance premium for individual customers based on their demographic and history. This can help insurance companies accurately price their policies, provide personalized quotes, and understand the factors driving premium variations.

### Data
This data set was provided by Kaggle:
https://www.kaggle.com/datasets/merishnasuwal/auto-insurance-churn-analysis-dataset?select=demographic.csv
The provided data asset is relational and consists of four distinct data files.

1. address.csv: contains address information

2. customer.csv: contains customer information.

3. demographic.csv: contains demographic data

4. termination.csv: includes customer termination information.

All data sets are linked using either ADDRESS_ID or INDIVIDUAL_ID. The ADDRESS_ID pertains to a specific postal service address, while the INDIVIDUAL_ID is unique to each individual. It is important to note that multiple customers may be assigned to the same address, and not all customers have demographic information available.

The data set includes 1,536,673 unique addresses and 2,280,321 unique customers, of which 2,112,579 have demographic information. Additionally, 269,259 customers cancelled their policies within the previous year. Finally, the customer information in this data set is fictituous although it refers information from Texas. 

In [1]:
#import packages

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
#load csv files

address = pd.read_csv("address.csv")
churn = pd.read_csv("autoinsurance_churn.csv")
customer = pd.read_csv("customer.csv")
demo = pd.read_csv("demographic.csv")
terminate = pd.read_csv("termination.csv")




### Data at a Glance

In [3]:
address.head(5)


,ADDRESS_ID,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,STATE,COUNTY
0,5.213011e+11,32.315803,-96.627896,8457 Wright Mountains Apt. 377,Ennis,TX,Ellis
1,5.213000e+11,NaN,NaN,082 Cline Mountains Apt. 353,Irving,TX,Dallas
2,5.213002e+11,32.806290,-96.779857,457 John Mills,Dallas,TX,Dallas
3,5.213013e+11,32.825737,-96.939687,5726 Barnett Meadow,Irving,TX,Dallas
4,5.213010e+11,32.867192,-96.715552,050 Nicholas Views,Dallas,TX,Dallas


In [4]:
churn.head(5)

,individual_id,address_id,curr_ann_amt,days_tenure,cust_orig_date,age_in_years,date_of_birth,latitude,longitude,city,state,county,income,has_children,length_of_residence,marital_status,home_market_value,home_owner,college_degree,good_credit,acct_suspd_date,Churn
0,2.213000e+11,5.213000e+11,818.877997,1454.0,2018-12-09,44,1978-06-23,32.578829,-96.305006,Kaufman,TX,Kaufman,22500.0,1.0,15.0,Married,50000 - 74999,1.0,1.0,1.0,NaN,0
1,2.213001e+11,5.213001e+11,974.199182,1795.0,2018-01-02,72,1950-05-30,32.732209,-97.000893,Grand Prairie,TX,Dallas,27500.0,0.0,2.0,Single,50000 - 74999,1.0,0.0,0.0,NaN,0
2,2.213007e+11,5.213002e+11,967.375112,4818.0,2009-09-23,55,1967-07-07,32.819777,-96.846938,Dallas,TX,Dallas,42500.0,0.0,10.0,Married,75000 - 99999,1.0,0.0,0.0,NaN,0
3,2.213016e+11,5.213006e+11,992.409561,130.0,2022-07-25,53,1969-05-25,32.684065,-97.162180,Arlington,TX,Tarrant,125000.0,1.0,6.0,Married,175000 - 199999,1.0,0.0,1.0,2021-12-22,1
4,2.213016e+11,5.213006e+11,784.633494,5896.0,2006-10-11,50,1972-09-25,32.751398,-97.376745,Fort Worth,TX,Tarrant,87500.0,1.0,4.0,Married,225000 - 249999,1.0,1.0,1.0,NaN,0


In [5]:
customer.head(5)

,INDIVIDUAL_ID,ADDRESS_ID,CURR_ANN_AMT,DAYS_TENURE,CUST_ORIG_DATE,AGE_IN_YEARS,DATE_OF_BIRTH,SOCIAL_SECURITY_NUMBER
0,2.213000e+11,5.213000e+11,818.877997,1454.0,2018-12-09,44.474,1978-06-23,608-XX-7640
1,2.213001e+11,5.213001e+11,974.199182,1795.0,2018-01-02,72.559,1950-05-30,342-XX-6908
2,2.213007e+11,5.213002e+11,967.375112,4818.0,2009-09-23,55.444,1967-07-07,240-XX-9224
3,2.213016e+11,5.213006e+11,992.409561,130.0,2022-07-25,53.558,1969-05-25,775-XX-6249
4,2.213016e+11,5.213006e+11,784.633494,5896.0,2006-10-11,50.220,1972-09-25,629-XX-7298


In [6]:
demo.head(5)

,INDIVIDUAL_ID,INCOME,HAS_CHILDREN,LENGTH_OF_RESIDENCE,MARITAL_STATUS,HOME_MARKET_VALUE,HOME_OWNER,COLLEGE_DEGREE,GOOD_CREDIT
0,2.213028e+11,125000.000,1.0,8.0,Single,300000 - 349999,1,1,1
1,2.213032e+11,42500.000,0.0,0.0,Single,NaN,0,0,0
2,2.213032e+11,27500.000,0.0,15.0,Married,75000 - 99999,1,0,1
3,2.213032e+11,80372.176,0.0,0.0,NaN,1000 - 24999,1,0,0
4,2.213032e+11,125000.000,0.0,0.0,NaN,NaN,0,0,1


In [7]:
terminate.head(5)

,INDIVIDUAL_ID,ACCT_SUSPD_DATE
0,2.213026e+11,2022-10-09
1,2.213028e+11,2022-04-24
2,2.213027e+11,2022-05-21
3,2.213002e+11,2022-04-27
4,2.213026e+11,2022-09-16


### Cleaning and Preparing Data

In [8]:
# change column names to lowercase

dfs = [address, churn, customer, demo, terminate]

for df in dfs:
    df.columns = df.columns.str.lower()

In [9]:
# check for missing values values
dfs_name = {'address':address, 'churn':churn, 'customer':customer, 'demo':demo, 'terminate':terminate}

for df_name, df in dfs_name.items():
    print(f"Missing values in {df_name}:")
    print(df.isnull().sum())
    print('\n')

Missing values in address:
address_id             0
latitude          129961
longitude         129961
street_address         0
city               15434
state                  0
county             15434
dtype: int64


Missing values in churn:
individual_id                0
address_id                   0
curr_ann_amt                 0
days_tenure                  0
cust_orig_date               0
age_in_years                 0
date_of_birth                0
latitude                253719
longitude               253719
city                     12067
state                        0
county                   12067
income                       0
has_children                 0
length_of_residence          0
marital_status               0
home_market_value        92286
home_owner                   0
college_degree               0
good_credit                  0
acct_suspd_date        1487453
churn                        0
dtype: int64


Missing values in customer:
individual_id                  1


There are many missing values in each data frame. Latitude and longitude have a high frequence of mising values. This might not be to critical for the analysis we will do later. Account suspention date also has a lot of missing values which could imply that all these accounts are still active. There are missing values in age but that can be populated since we know the customers date of birth in `customer` df. Since most data frames have a `customer_id`, it would be useful to use it as our index.

In [10]:
# check row with missing individual_id in customer df

customer[customer['individual_id'].isnull()]

# remove this row to convert dtype of individual_id to int64 to use as index for merged dfs

customer = customer.dropna(subset = ['individual_id'])

In [11]:
# change individual_id from float64 to int64

dfs_iid = [terminate, demo, customer, churn]

for df in dfs_iid:
    df['individual_id'] = df['individual_id'].astype('int64')

C:\Users\mende\AppData\Local\Temp\ipykernel_10620\148917070.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['individual_id'] = df['individual_id'].astype('int64')


Before we continue analyzing our data, merging our dataframes into one dataframe will allows us to purposefully fill missing values, engineer valuable features, remove unnecessary data, etc. We will leave the `terminate` and `demo` dfs out since this information is captured in the `churn` df. 

In [12]:
# merge customer and address df on address_id
ctmr_add = customer.merge(address, on = 'address_id')

ctmr_add.head()

,individual_id,address_id,curr_ann_amt,days_tenure,cust_orig_date,age_in_years,date_of_birth,social_security_number,latitude,longitude,street_address,city,state,county
0,221300017872,5.213000e+11,818.877997,1454.0,2018-12-09,44.474,1978-06-23,608-XX-7640,32.578829,-96.305006,52966 Welch Crescent,Kaufman,TX,Kaufman
1,221300134410,5.213001e+11,974.199182,1795.0,2018-01-02,72.559,1950-05-30,342-XX-6908,32.732209,-97.000893,46887 Lawrence Green,Grand Prairie,TX,Dallas
2,221300134409,5.213001e+11,670.603798,1097.0,2019-12-01,51.305,1971-08-26,418-XX-4007,32.732209,-97.000893,46887 Lawrence Green,Grand Prairie,TX,Dallas
3,221300673028,5.213002e+11,967.375112,4818.0,2009-09-23,55.444,1967-07-07,240-XX-9224,32.819777,-96.846938,787 Daniel Mews Suite 806,Dallas,TX,Dallas
4,221300673027,5.213002e+11,923.707801,4539.0,2010-06-29,38.639,1984-04-22,175-XX-7906,32.819777,-96.846938,787 Daniel Mews Suite 806,Dallas,TX,Dallas


In [41]:
# merge the churn df

cust_data = pd.concat([ctmr_add, churn], axis = 1)

# drop duplicate column names NOTE: this is ok since columns contained identical information

cust_data = cust_data.loc[:, ~ cust_data.columns.duplicated()]

cust_data.head()

,individual_id,address_id,curr_ann_amt,days_tenure,cust_orig_date,age_in_years,date_of_birth,social_security_number,latitude,longitude,street_address,city,state,county,income,has_children,length_of_residence,marital_status,home_market_value,home_owner,college_degree,good_credit,acct_suspd_date,churn
0,221300017872,5.213000e+11,818.877997,1454.0,2018-12-09,44.474,1978-06-23,608-XX-7640,32.578829,-96.305006,52966 Welch Crescent,Kaufman,TX,Kaufman,22500.0,1.0,15.0,Married,50000 - 74999,1.0,1.0,1.0,NaN,0.0
1,221300134410,5.213001e+11,974.199182,1795.0,2018-01-02,72.559,1950-05-30,342-XX-6908,32.732209,-97.000893,46887 Lawrence Green,Grand Prairie,TX,Dallas,27500.0,0.0,2.0,Single,50000 - 74999,1.0,0.0,0.0,NaN,0.0
2,221300134409,5.213001e+11,670.603798,1097.0,2019-12-01,51.305,1971-08-26,418-XX-4007,32.732209,-97.000893,46887 Lawrence Green,Grand Prairie,TX,Dallas,42500.0,0.0,10.0,Married,75000 - 99999,1.0,0.0,0.0,NaN,0.0
3,221300673028,5.213002e+11,967.375112,4818.0,2009-09-23,55.444,1967-07-07,240-XX-9224,32.819777,-96.846938,787 Daniel Mews Suite 806,Dallas,TX,Dallas,125000.0,1.0,6.0,Married,175000 - 199999,1.0,0.0,1.0,2021-12-22,1.0
4,221300673027,5.213002e+11,923.707801,4539.0,2010-06-29,38.639,1984-04-22,175-XX-7906,32.819777,-96.846938,787 Daniel Mews Suite 806,Dallas,TX,Dallas,87500.0,1.0,4.0,Married,225000 - 249999,1.0,1.0,1.0,NaN,0.0


In [55]:
cust_data.isnull().sum()

individual_id                   0
address_id                      0
curr_ann_amt                    0
days_tenure                     0
cust_orig_date                  0
age_in_years               167765
date_of_birth                   0
social_security_number          0
latitude                   471491
longitude                  471491
street_address                  0
city                        17008
state                           0
county                      17008
income                     599411
has_children               599411
length_of_residence        599411
marital_status             599411
home_market_value            6959
home_owner                 599411
college_degree             599411
good_credit                599411
acct_suspd_date           2086864
churn                      599411
dtype: int64

In [59]:
cust_data[cust_data['county'].isnull()].head()

,individual_id,address_id,curr_ann_amt,days_tenure,cust_orig_date,age_in_years,date_of_birth,social_security_number,latitude,longitude,street_address,city,state,county,income,has_children,length_of_residence,marital_status,home_market_value,home_owner,college_degree,good_credit,acct_suspd_date,churn
1129,221303291761,5.213015e+11,655.653460,1960.0,2017-07-21,39.387,1983-07-24,610-XX-3821,NaN,NaN,339 Nichols View,NaN,TX,NaN,62500.0,1.0,10.0,Married,1000 - 24999,1.0,0.0,1.0,NaN,0.0
2891,221302753150,5.213010e+11,1038.839760,6291.0,2005-09-11,47.220,1975-09-25,552-XX-1989,32.760158,-96.803220,83756 Gabriel Flat Apt. 395,NaN,TX,NaN,27500.0,0.0,4.0,Single,250000 - 274999,0.0,0.0,1.0,NaN,0.0
2892,221302753149,5.213010e+11,1248.270650,87.0,2022-09-06,41.725,1981-03-22,717-XX-6244,32.760158,-96.803220,83756 Gabriel Flat Apt. 395,NaN,TX,NaN,62500.0,0.0,6.0,Single,400000 - 449999,1.0,0.0,1.0,NaN,0.0
4527,221302754491,5.213010e+11,738.078569,2704.0,2015-07-08,67.305,1955-08-30,188-XX-1002,32.651159,-97.372223,5748 Jackson Plain Apt. 177,NaN,TX,NaN,27500.0,1.0,15.0,Single,75000 - 99999,1.0,0.0,1.0,NaN,0.0
4953,221303302608,5.213015e+11,935.916115,1839.0,2017-11-19,68.307,1954-08-29,438-XX-5372,NaN,NaN,332 Morris Pass Suite 720,NaN,TX,NaN,87500.0,1.0,5.0,Married,125000 - 149999,1.0,1.0,1.0,NaN,0.0


Many observations that are missing values in `city` and `county` have latitude and longitude information. We can use this information to fill the missing values and minimize the amount of observations we have to drop because of missing values. 

In [54]:
# fill missing value in home_market_value with median of county

mode_home_val = cust_data.groupby('city')['home_market_value'].transform(lambda x: x.mode()[0])

cust_data['home_market_value'] = cust_data['home_market_value'].fillna(mode_home_val)

### Data Exploration

In [46]:
cust_data.shape

(2280320, 24)

In [56]:
cust_data.isnull().sum()

individual_id                   0
address_id                      0
curr_ann_amt                    0
days_tenure                     0
cust_orig_date                  0
age_in_years               167765
date_of_birth                   0
social_security_number          0
latitude                   471491
longitude                  471491
street_address                  0
city                        17008
state                           0
county                      17008
income                     599411
has_children               599411
length_of_residence        599411
marital_status             599411
home_market_value            6959
home_owner                 599411
college_degree             599411
good_credit                599411
acct_suspd_date           2086864
churn                      599411
dtype: int64

In [44]:
# Insurance policy counts by County and City
pd.set_option("display.max_rows", None)

print(cust_data['county'].value_counts())
print('\n')
print(cust_data['city'].value_counts())

Dallas      865587
Tarrant     658923
Collin      291553
Denton      248770
Ellis        57850
Kaufman      41896
Parker       31827
Rockwall     30634
Johnson      30616
Hunt          2273
Cooke         1994
Navarro        743
Grayson        552
Hill            94
Name: county, dtype: int64


Dallas                   435836
Fort Worth               303494
Arlington                134094
Plano                    110257
Garland                   82254
Irving                    73577
Grand Prairie             58530
Mckinney                  54700
Mesquite                  50804
Denton                    46573
Frisco                    46441
Carrollton                45631
Lewisville                39427
Richardson                38569
Keller                    35602
Allen                     35409
North Richland Hills      25080
Flower Mound              24752
Burleson                  23816
Euless                    23133
Mansfield                 22966
Rockwall                  22579
R

## Notes:
* All dfs except address contain individual_id. We can merge on this column. individual_id would be a good index for a merged df.
* Address, churn and customer contain an address_id.

## To do:
* [complete] in churn dataset change the Churn column name to lowercase
* [complete] in address+customer+demo+terminate dataset, change colnames to lowercase
* [complete] merge dfs to a single df
* use `latitude` and `longitude` to fill missing value in `city` and `county`
* analyze dfs, clean, remove/replace NAs